In [2]:
# stretching the cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# cuda settings
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.50)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Reading then tfrecords

In [9]:
def read_training_data(data_path='train.tfrecords'):
    data_path = "train.tfrecords"
    feature = {'train/image':tf.FixedLenFeature([],tf.string), 'train/label':tf.FixedLenFeature([],tf.int64)}
    filename_queue = tf.train.string_input_producer([data_path],num_epochs=10)

    reader = tf.TFRecordReader()
    _,serialzed_example = reader.read(filename_queue)

    features = tf.parse_single_example(serialzed_example,features=feature)

    image = tf.decode_raw(features["train/image"],tf.float32)
    label = tf.cast(features["train/label"],tf.int32)

    image = tf.reshape(image,[224,224,3])
    label = tf.one_hot(label,depth=8)
    images,labels = tf.train.shuffle_batch([image,label],batch_size=10,capacity=100,num_threads=1,min_after_dequeue=10)

    return (images,labels)
    
# init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
# with tf.Session() as sess:
#     sess.run(init_op)

#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)

#     for i in range(1):
#         img,lbl = sess.run([images,labels])
#         print(img.shape,lbl.shape)        
#     coord.request_stop()

#     coord.join(threads)
#     sess.close()

        
# with tf.Session() as sess:
#     images,labels = read_training_data()
#     init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())  
#     sess.run(init_op)
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)
#     for i in range(1):
#         i,l = sess.run([images,labels])
#         i = i.astype(np.uint8)
#         print(i.shape,l.shape)
#         plt.imshow(i[99])
#         plt.title(l[99])
#         plt.show()
        

#     coord.request_stop()

#     coord.join(threads)
#     sess.close()


In [10]:
def read_validation_data(data_path='valid.tfrecords'):
    data_path = "valid.tfrecords"
    feature = {'valid/image':tf.FixedLenFeature([],tf.string), 'valid/label':tf.FixedLenFeature([],tf.int64)}
    filename_queue = tf.train.string_input_producer([data_path],num_epochs=2)

    reader = tf.TFRecordReader()
    _,serialzed_example = reader.read(filename_queue)

    features = tf.parse_single_example(serialzed_example,features=feature)

    image = tf.decode_raw(features["valid/image"],tf.float32)
    label = tf.cast(features["valid/label"],tf.int32)

    image = tf.reshape(image,[224,224,3])
    label = tf.one_hot(label,depth=8)
    images,labels = tf.train.shuffle_batch([image,label],batch_size=100,capacity=100,num_threads=1,min_after_dequeue=10)

    return (images,labels)
    

## Creating the default graph

In [11]:
tf.reset_default_graph()

print("Loading default graph.....")
graph = tf.Graph()
with graph.as_default():
    
    model_path = "./vgg_16.ckpt"
    assert(os.path.isfile(model_path))

    vgg = slim.nets.vgg
    image_size = vgg.vgg_16.default_image_size
    num_classes = 8
    
    x = tf.placeholder(tf.float32,shape=(None,image_size,image_size,3))
    y = tf.placeholder(tf.int32,shape=(None,num_classes))
    with slim.arg_scope(vgg.vgg_arg_scope(weight_decay=0.0001)):
        logits,end_points = vgg.vgg_16(x,num_classes=num_classes,is_training=True)
        


    variables_to_restore = slim.get_variables_to_restore(exclude=['vgg_16/fc8'])
    init_fn = slim.assign_from_checkpoint_fn(model_path,variables_to_restore)

    fc8_variables = slim.get_variables('vgg_16/fc8')
    fc8_init = tf.variables_initializer(fc8_variables)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y))

    fc8_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    fc8_train_op = fc8_optimizer.minimize(loss,var_list=fc8_variables)

    full_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
    full_train_op = full_optimizer.minimize(loss)

    correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    # actual = tf.argmax(y,1)
    prediction = tf.argmax(logits,1)
        
    # tf.get_default_graph().finalize()  
    
    print("\nFinalized the graph ... !")


Loading default graph.....

Finalized the graph ... !


In [12]:
assert logits.graph == graph

# Training

In [13]:
with tf.Session(graph=graph,config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    images,labels = read_training_data()
    valid_images,valid_labels = read_validation_data()
    
    init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())  
    sess.run(init_op)
    
    init_fn(sess)
    sess.run(fc8_init)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    print("Started Training ... !")

#     epoch=0
#     try:
#         while not coord.should_stop():
#             img,lbl = sess.run([images,labels])
#             img = img.astype(np.uint8)
#             # print("Next batch : {}".format(img.shape,lbl.shape))
#             sess.run(fc8_train_op,feed_dict={x:img,y:lbl})
#             print("Loss : {:.2f}".format(sess.run(loss,feed_dict={x:img,y:lbl})))
#             epoch+=1
#     except tf.errors.OutOfRangeError as e:
#         coord.request_stop(e)

    for epoch in range(10):
        img,lbl = sess.run([images,labels])
        img = img.astype(np.uint8)
        # print("Next batch : {}".format(img.shape,lbl.shape))
        sess.run(fc8_train_op,feed_dict={x:img,y:lbl})
        
    val_img,val_lbl = sess.run([valid_images,valid_labels])
    val_img = val_img.astype(np.uint8)
    print("Validation Accuracy after retraining for {} epoch : {:.2f}".format(epoch+1,sess.run(accuracy,feed_dict={x:val_img,y:val_lbl})))


    epoch=0
    try:
        while not coord.should_stop():
            img,lbl = sess.run([images,labels])
            sess.run(full_train_op,feed_dict={x:img,y:lbl})
            print(sess.run(loss,feed_dict={x:img,y:lbl}))
            epoch+=1
    except tf.errors.OutOfRangeError as e:
        coord.request_stop(e)
        
    val_img,val_lbl = sess.run([valid_images,valid_labels])
    print("Validation accuracy after fune tuning for epoch {} : {}".format(epoch,sess.run(accuracy,feed_dict={x:val_img,y:val_lbl})))  

    coord.request_stop()
    coord.join(threads)
    sess.close()


INFO:tensorflow:Restoring parameters from ./vgg_16.ckpt
Started Training ... !
Validation Accuracy after retraining for 10 epoch : 0.30
0.46884537
1.5828413
2.3210912
2.2102864
2.2987585
1.2344649
2.1070848
3.1716313
1.7021847
2.767602
2.4365942
1.8044657
2.1472318
1.8912977
2.0627327
1.0303156
2.0095344
2.257977
2.6861587
2.2281647
1.9058971
2.0741856
2.0520089
2.3768961
1.4404333
1.4544898
2.0863311
1.0653149
0.9906745
2.4846907
2.5362048
3.377974
3.7686772
1.5588417
2.1459813
1.6524994
2.5544462
1.9462707
1.7321975
2.8573623
3.7788315
4.263249
2.495042
3.1601462
2.0745957
2.1103022
1.700491
1.6294644
0.64711
2.2953496
3.3453434
3.0266688
1.7607622
1.8391562
1.796822
2.1103806
0.9485003
1.0519779
0.29233357
2.2365055
2.4635212
2.7278972
2.8366249
3.3455043
1.7363584
1.2423923
1.730862
1.6742408
1.2506138
1.5817688
1.5059102
1.4719489
1.7848917
2.1006875
1.4042399
1.5329721
0.72922754
0.66675574
1.0329626
1.5857325
1.5467293
1.5266397
2.1878617
1.2033187
1.6394527
1.0183895
1.2504276


0.010705427
0.0003691101
0.3689828
0.476052
0.47964525
0.031044146
0.51103085
0.17868923
0.28634658
0.03319425
0.43149298
0.2618402
0.31884742
Validation accuracy after fune tuning for epoch 790 : 0.8199999928474426


# Experiment